# Assistant

Assistant는 대화기록을 보고, 다음에 Agent가 취해야할 가장 적절한 행동을 선택하는 것을 목표로 한다.

Agent의 행동은 9가지로 구성된다.
1. 시작하기: 자기 소개로 대화를 시작하세요. 정중함, 존중, 전문적인 어조를 유지하세요.
2. 분석: 와인 추천을 위한 고객의 니즈를 파악합니다. 와인 데이터베이스 도구는 사용할 수 없습니다. 고객이 와인을 즐길 때, 와인과 함께 무엇을 먹을지, 원하는 가격대에 대해 물어봅니다. 한 번에 한 가지 질문만 하세요.
3. 가격대 확인: 고객이 선호하는 가격대를 묻습니다. 다시 한 번 말씀드리지만, 이를 위한 도구는 제공되지 않습니다. 그러나 고객의 선호도와 가격대를 알고 있다면 도구를 사용하여 가장 적합한 와인 세 가지를 검색하고 와인을 추천하세요. 각 와인 추천은 비비노 링크, 가격, 등급, 와인 유형, 풍미 설명 및 이미지가 포함된 목록 형식의 제품 카드 형태로 이루어져야 합니다. 데이터베이스에 있는 와인만 추천에 사용하세요. 데이터베이스에 적합한 와인이 없는 경우 고객에게 알려주세요. 추천을 한 후 고객이 추천한 와인을 좋아하는지 문의합니다.
4. 와인 추천: 고객의 요구와 가격대에 따라 가장 적합한 와인 3가지를 제안합니다. 추천하기 전에 고객이 와인을 즐길 상황, 와인과 함께 먹을 음식, 원하는 가격대를 파악해야 합니다. 각 와인 추천은 비비노 링크, 가격, 등급, 와인 유형, 풍미 설명 및 이미지가 포함된 목록 형식의 제품 카드 형태로 이루어져야 합니다. 데이터베이스에 있는 와인만 추천에 사용하세요. 데이터베이스에 적합한 와인이 없는 경우 고객에게 알려주세요. 추천을 한 후 고객이 추천한 와인을 좋아하는지 문의합니다.
5. 판매: 고객이 추천 와인을 승인하면 자세한 설명을 제공합니다. 비비노 링크, 가격, 등급, 와인 종류, 풍미 설명 및 이미지가 포함된 목록 형식의 제품 카드를 제공합니다.
6. 위치 제안: 고객의 위치와 상황에 따라 와인 바를 추천하세요. 추천하기 전에 항상 지도 도구를 사용하여 고객이 선호하는 위치의 지역을 찾아보세요. 그런 다음 와인 바 데이터베이스 도구를 사용하여 적합한 와인 바를 찾습니다. 와인 바의 이름, URL, 등급, 주소, 메뉴, 영업시간, 휴일, 전화, 요약, 이미지가 포함된 이미지를 목록 형식으로 제품 카드에 입력합니다. 추천에는 데이터베이스에 있는 와인 바만 사용하세요. 데이터베이스에 적합한 와인 바가 없는 경우 고객에게 알려주세요. 추천을 한 후 고객이 추천한 와인을 좋아하는지 문의합니다.
7. 대화 마무리하기: 고객의 의견에 적절하게 응답하여 대화를 마무리합니다.
8. 질문 답변: 이 단계에서는 고객의 문의에 답변합니다. 가능한 경우 검색 도구 또는 와인 데이터베이스 도구를 사용하여 구체적인 답변을 제공하세요. 가능한 자세하게 답변 설명합니다.
9. 기타 상황: 상황이 1~8단계 중 어느 단계에도 해당하지 않는 경우 이 단계를 사용합니다. 

### API 키 불러오기

In [1]:
import os
import configparser
import time
import copy

In [2]:
config = configparser.ConfigParser()
config.read('../secrets.ini')

['../secrets.ini']

In [3]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
serper_api_key = config['SERPER']['SERPER_API_KEY']
serp_api_key = config['SERPAPI']['SERPAPI_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})
os.environ.update({'SERPER_API_KEY': serper_api_key})
os.environ.update({'SERPAPI_API_KEY': serp_api_key})

### Get Stage Analyzer(Assistant) Prompt

In [4]:
from typing import List, Union
import re
import json

import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.prompts import load_prompt, BaseChatPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.vectorstores import DocArrayInMemorySearch, Chroma

프롬프트를 선언하는 방식은 파일로 부터 불러오거나 직접 선언할 수 있습니다. 아래에는 두 방식 모두 구현되어 있습니다.
- 직접 선언하기 위해서는 prompt_template을 먼저 작성해야합니다.
- 파일로부터 불러오는 것은 ./templates/stage_analyzer_inception_prompt_template.json 파일을 불러오는 것입니다.

In [5]:
# 직접 프롬프트 선언하기위한 템플릿 작성
stage_analyzer_inception_prompt_template = """
You are the assistant who helps determine which stage the agent should move to in response to the customer's last saying.
Here are the nine conversation stages the agent will progress through. Make sure to consider the entire context.
Conversation stages:
1. Introduce: Begin the conversation with a simple greeting and introduce yourself without addressing any request.
2. Analyze needs: If the customer is looking for wine or wine bar recommendations, ask questions to understand their needs.
3. Ask price range: Before recommending a wine, ask your customers what price range they're looking for.
4. Recommend a wine: After identifying the customer's needs and price range, recommend a suitable wine. This includes providing alternate recommendations if requested.
5. Sales: If the customer expresses interest in a wine or wishes to make a purchase, describe the product in detail and encourage them to finalize their decision.
6. Suggest a location: Recommend a wine bar near the customer's desired location. You may need to ask questions to determine the location.
7. Wrap up the conversation: Thank the customer, agree with them, or provide a closing statement.
8. Questions and answers: Provide detailed and accurate answers to questions about topics other than wine or location recommendations, such as wine terminology, cellar suggestions, wine knowledge, etc.
9. None of the given stages: Respond to questions that don't fit into any of the stages 1-8 or are unrelated to wine.

Please follow this code of conduct before answering: 
Before making a wine recommendation (Stage 4), ensure you analyze the customer's needs (Stage 2) and identify their price range (Stage 3).
When recommending wines (Stage 4), confirm that you know the customer's preferred price range (Stage 3). If not, inquire about the price range (Stage 3).
If the customer suggests a desired price range or you already know price range, you should skip asking the price range (Stage 3).
Make sure asking price range(Stage 3) should appear only once in the "stage_history".
To comprehend the context of the conversation, refer to the "stage_history".
Please be mindful of the frequency of "needs analysis (Stage 2)" appearing in the "stage_history," ensuring that "needs analysis (Stage 2)" appears approximately two times within a window of size 4 in the "stage_history".
Considering the conversation_history and last_user_saying, think carefully, and provide your answer step-by-step. 
When a customer requests a recommendation, be sure to clearly understand their needs. 
Provide answers in numbers only, with no words.

Below is three examples:
===
stage history: 2, 2, 3
---
conversation_history:
User: I'm having a birthday party with my friends, can you recommend a good wine?<END_OF_TURN>
Agent: Sure, sure. Do you have a favorite type of wine? <END_OF_TURN>
User: I like wines with some body.<END_OF_TURN>
Agent: Okay, so do you have a particular country of origin in mind?<END_OF_TURN>
User: I like French wines, but other countries are fine.<END_OF_TURN>
Agent: Very good, so what price range do you have in mind?<END_OF_TURN>
---
last_user_saying: I'm thinking around 100,000 won. If it's a good wine, I don't mind paying a little more.<END_OF_TURN>
===
Answer:4

===
stage history: 2
---
conversation_history: 
User: Can you recommend a good wine for a beginner? <END_OF_TURN>
Agent: Yes, that would be great. What kind of atmosphere do you prefer when you enjoy wine? For example, a party with friends, a quiet dinner, or just relaxing by yourself.<END_OF_TURN>
---
last_user_saying: Please recommend a wine for a quiet dinner. <END_OF_TURN>
===
Answer:2

===
stage_history : 
---
conversation_history: 
 
---
last_user_saying: Can you recommend a good wine for a beginner? <END_OF_TURN>

===
Answer:2

===
stage_history : {stage_history}
---
conversation_history: 
{conversation_history} 
---
last_user_saying: {last_user_saying}
===
Answer:
"""

In [7]:
# 직접 프롬프트 선언하기
stage_analyzer_inception_prompt = PromptTemplate(
    input_variables=["conversation_history", "stage_history", "last_user_saying"], 
    template=stage_analyzer_inception_prompt_template,
)

In [10]:
# 파일로 부터 프롬프트 불러오기
stage_analyzer_inception_prompt = load_prompt("../model/templates/assistant_prompt_template.json")

FileNotFoundError: [Errno 2] No such file or directory: 'model\\templates\\assistant_prompt_template.txt'

아래와 같이 format 메서드로 프롬프트를 확인할 수 있습니다.

In [18]:
example_conversation_history = """
User: 안녕하세요. <END_OF_TURN>
이우선: 무엇을 도와드릴까요? <END_OF_TURN>
User: 와인 추천해주세요. <END_OF_TURN>
이우선: 어떤 행사나 기념일을 위해 와인을 찾으시는지 알려주실 수 있으신가요? <END_OF_TURN>
"""
example_last_user_saying = "이번주에 결혼기념일이 있어서요. <END_OF_TURN>"
example_stage_history = "1, 2"
example_answer = "2"
# 여기서 우리는 에이전트가 와인 추천을 위해 유저의 더 많은 정보를 얻기를 원한다. 따라서 2(Needs Analysis)가 답변으로 나오길 원한다.

In [19]:
print(
    stage_analyzer_inception_prompt.format(
        conversation_history=example_conversation_history,
        stage_history=example_stage_history,
        last_user_saying=example_last_user_saying
    )
)


You are the assistant who helps determine which stage the agent should move to in response to the customer's last saying.
Here are the nine conversation stages the agent will progress through. Make sure to consider the entire context.
Conversation stages:
1. Introduce: Begin the conversation with a simple greeting and introduce yourself without addressing any request.
2. Analyze needs: If the customer is looking for wine or wine bar recommendations, ask questions to understand their needs.
3. Ask price range: Before recommending a wine, ask your customers what price range they're looking for.
4. Recommend a wine: After identifying the customer's needs and price range, recommend a suitable wine. This includes providing alternate recommendations if requested.
5. Sales: If the customer expresses interest in a wine or wishes to make a purchase, describe the product in detail and encourage them to finalize their decision.
6. Suggest a location: Recommend a wine bar near the customer's desi

In [15]:
# 랭체인 모델 선언, 랭체인은 언어모델과 프롬프트로 구성됩니다.
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)
stage_analyzer_chain = LLMChain(
    llm=llm,
    prompt=stage_analyzer_inception_prompt, 
    verbose=True, # 과정을 출력할지
    output_key="stage_number" # 출력값의 변수명
    )

Chain을 실행하기 위해서는 run 메서드를 실행한다.

In [20]:
stage_number = stage_analyzer_chain.run(
    {
        'conversation_history': example_conversation_history, 
        'stage_history': example_stage_history,
        'last_user_saying': example_last_user_saying
    }
)

결과를 확인해보면 원하는 결과(2. Needs Analysis)가 나오는 것을 확인할 수 있다.

In [21]:
print(stage_number)

2
